# 2019 OK JPL Horizon vs OpenOrb comparison

## Assumptions/ accountability-sourced documentation:

* Values for chosen orbit found here: https://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2019%20OK
* Location: I11, Gemini South
* Time: 2010/01/01 to 2020/01/01, midnights

## Current questions/ issues:

* Do I need to cite things in-code? nah, but track examples/ code snippets
* Do we need a maximum difference function or class? What do I need to know about OOP for class things?
* Key error: 'R.A. (ICRF)' ??? what do ??? report???
* dynmodel = N still works, yeah?

In [5]:
%matplotlib inline

import sys
import numpy as np
import astropy as ap
import astropy.units as u
from astropy.table import QTable
import matplotlib.pyplot as plt
import astroquery as aq
from astroquery.jplhorizons import Horizons

In [58]:
# "Global" variables: times, location, object
start_time = '2010-01-01T00:00:00'
stop_time = '2020-01-01T00:00:00'
element_time = ap.time.Time(start_time).jd
obj_id = '2019 OK'
Gemini_S = 'I11@399'

In [62]:
import pyoorb as oo
oo.pyoorb.oorb_init()

0

In [87]:
el_obj = Horizons(id=obj_id, location= '500@399',
               epochs=element_time)
el = el_obj.elements()
dat = np.array(el)


TypeError: 'numpy.ndarray' object is not callable

In [19]:
'''
This is where in-line things happen. 
Gfortran version 4.8.6

Useful bits from docs:
make ephem
oorb --task=propagation --epoch-mjd-utc=DATE --orb-in=ORBITFILEIN --orb-out=ORBITFILEOUT
oorb --task=ephemeris --orb-in=ORBBITFILEOUT
pyoorb.oorb_ephemeris_basic over pyoorb.oorb_ephemeris_full
Below is an example copied from--> https://github.com/oorb/oorb/tree/master/python#ephemeris-computation
It felt useful to study, I intend to shuffle things around to get to what I need from it
'''
# orbits is where selected element values go that we got from JPL
orbits = np.array([[0, 1.46905, 0.33435, np.deg2rad(14.3024),
                    np.deg2rad(224.513), np.deg2rad(27.5419),
                    np.deg2rad(324.697), 3, 51544.5, 1, 12.5, 0.15]],
                    dtype=np.double, order='F')

epoch = np.array([51232.23, 3], dtype=np.double, order='F')
orb, err = oo.pyoorb.oorb_propagation(in_orbits=orbits,
                                     in_epoch=epoch,
                                    in_dynmodel='N')
                                    
mjds = np.arange(51232, 51233, 1/24) # figure out what this is
epochs = np.array(list(zip(mjds, [1]*len(mjds))), dtype=np.double, order='F')
eph, err = oo.pyoorb.oorb_ephemeris_full(in_orbits=orbits,
                                         in_obscode='568',
                                         in_date_ephems=epochs,
                                         in_dynmodel='N')
print(err, eph)

0 [[[ 5.12320000e+04  2.97420305e+01  9.39898382e+00  4.54411204e-01
    1.15967356e-01  2.55335063e+01  5.70687690e+01  1.92629817e+00
    2.27595666e+00  1.68784924e+01  7.56834978e+01  3.09658224e+01
   -2.57079287e+00 -1.22968568e+02 -2.57187230e+00  5.65262567e+01
   -3.03728791e+00 -9.74081340e+01 -3.03836734e+00  5.67373413e+01
    5.36581121e+01  2.79270091e+01  4.93916942e-01  3.23198414e+01
    1.06096650e+00  1.60454266e+00 -1.02066546e-01 -7.60331277e-03
    6.44843751e-03 -2.53113453e-03 -8.88643901e-01  4.34680947e-01
    1.86372479e-05  1.64854703e+02]
  [ 5.12320417e+04  2.97612057e+01  9.40381070e+00  4.53732760e-01
    1.15720650e-01  2.55263237e+01  5.70468341e+01  1.92635305e+00
    2.27643567e+00  1.68787982e+01  7.56922196e+01  3.09852619e+01
   -2.57280325e+00 -1.22990606e+02 -2.57364094e+00  5.65385408e+01
   -3.04034223e+00 -9.74373272e+01 -3.04117992e+00  7.00074489e+01
    4.35947788e+01  4.16228206e+01  4.97711138e-01  3.27693280e+01
    1.06064968e+00  1.60

# "Final" Draft Things Below This Line

In [58]:
# "Global" variables: times, location, object
start_time = '2010-01-01T00:00:00'
stop_time = '2020-01-01T00:00:00'
element_time = ap.time.Time(start_time).jd
obj_id = '2019 OK'
Gemini_S = 'I11@399'

## JPL-Sourced Values

In [60]:
# 2019 OK elements. NB: location is at center of Earth bc elements must be object-centered
el_obj = Horizons(id=obj_id, location= '500@399',
               epochs=element_time)
el = el_obj.elements()
print(el)

targetname datetime_jd ...           Q                     P          
   ---          d      ...           AU                    d          
---------- ----------- ... --------------------- ---------------------
 (2019 OK)   2455197.5 ... 9.999999999999998e+99 9.999999999999998e+99


In [61]:
# 2019 OK ephem
ephem_obj = Horizons(id= obj_id, location= Gemini_S,
               epochs={'start': start_time, 'stop':stop_time,
                      'step':'1d'})
eph = ephem_obj.ephemerides()
print(eph)

targetname    datetime_str   datetime_jd  H   ... alpha_true  PABLon   PABLat
   ---            ---             d      mag  ...    deg       deg      deg  
---------- ----------------- ----------- ---- ... ---------- -------- -------
 (2019 OK) 2010-Jan-01 00:00   2455197.5 23.3 ...    12.1727 227.5929 -1.2841
 (2019 OK) 2010-Jan-02 00:00   2455198.5 23.3 ...    12.3271 227.7476 -1.2858
 (2019 OK) 2010-Jan-03 00:00   2455199.5 23.3 ...    12.4797 227.9013 -1.2876
 (2019 OK) 2010-Jan-04 00:00   2455200.5 23.3 ...    12.6304 228.0541 -1.2894
 (2019 OK) 2010-Jan-05 00:00   2455201.5 23.3 ...    12.7792 228.2058 -1.2913
 (2019 OK) 2010-Jan-06 00:00   2455202.5 23.3 ...     12.926 228.3565 -1.2931
 (2019 OK) 2010-Jan-07 00:00   2455203.5 23.3 ...    13.0709 228.5061  -1.295
 (2019 OK) 2010-Jan-08 00:00   2455204.5 23.3 ...    13.2137 228.6547 -1.2969
 (2019 OK) 2010-Jan-09 00:00   2455205.5 23.3 ...    13.3544 228.8022 -1.2989
 (2019 OK) 2010-Jan-10 00:00   2455206.5 23.3 ...     13.493 228

## OpenOrb-Sourced Values

## Difference metric

In [20]:
# difference function

def max_diff_elements(var1, var2):
    
    # don't forget to keep the 360deg problem in mind
    
    return RA_diff, DEC_diff


def max_diff_ephem(var1, var2):
    
    # same 360 thing applies
    
    return RA_diff, DEC_diff